# Dilepton analysis - handling the MC background


### <font color='red'>Not a finished example!!</font>  
This example is only finished up to and including section 2. 

In [1]:
#include <iostream>
#include <string>
#include <stdio.h>

In [2]:
%jsroot on

## 1. Reading the dataset

In [3]:
TChain *dataset = new TChain("mini"); 

A list of all the background samples and their IDs can be found in **Background_samples.txt**. We read that list, and add all the samples to the TChain. We also (for later convenience) make a vector containing the dataset IDs. 

In [4]:
TString sample; 
TString path; 
vector<Int_t> dataset_IDs;
Int_t DSID;

In [5]:
ifstream infile("Background_samples.txt");

In [7]:
infile.clear();
infile.seekg(0, ios::beg);  // Start at the beginning of the file
dataset->Reset(); 
while(infile >> sample >> DSID){
    path = "DataSamples/MC/"+sample; // Specify path to the samples 
    dataset->Add(path);  
    dataset_IDs.push_back(DSID);
}

Define the variables you want to include in the analysis, and link them to branches in the TTree.

In [8]:
const int vs = 5; 

Int_t lepton_n = -1, lepton_charge[vs], lepton_type[vs], ID; 

Float_t lepton_pt[vs], lepton_E[vs], lepton_phi[vs], lepton_eta[vs], MET;

In [9]:
dataset->SetBranchAddress("lep_n",      &lepton_n);
dataset->SetBranchAddress("lep_charge", &lepton_charge);
dataset->SetBranchAddress("lep_type",   &lepton_type);
dataset->SetBranchAddress("lep_pt",     &lepton_pt);
dataset->SetBranchAddress("lep_eta",    &lepton_eta);
dataset->SetBranchAddress("lep_phi",    &lepton_phi);
dataset->SetBranchAddress("lep_E",      &lepton_E);
dataset->SetBranchAddress("met_et",     &MET); 
dataset->SetBranchAddress("channelNumber", &ID);

## 2. Making (a lot of) histograms

Now that we have read our dataset we want to start analyzing the data. To do so we need to put the data into histograms. For reasons that will become clear later in the analysis we must (for each variable) make one histogram per dataset ID. (We have 31 background samples, so if we want to study 10 variables we have to make 310 histograms!) A very elegant way of dealing with all these histograms is by using [map](http://www.cplusplus.com/reference/map/map/)s (the C$++$ equivalent of Python dictionaries). Below we define one map for each variable. Here the *key values* are the dataset IDs, while the *mapped values* are the histograms.   

In [10]:
map<Int_t, TH1*> hist_mll; 
map<Int_t, TH1*> hist_lep_pt; 
map<Int_t, TH1*> hist_met;

In [11]:
for(const auto & i:dataset_IDs){
    hist_mll[i] = new TH1F(); 
    hist_lep_pt[i] = new TH1F(); 
    hist_met[i] = new TH1F();
}

In [12]:
for(const auto & i:dataset_IDs){
    hist_mll[i]->SetNameTitle("hist_mll", "Invariant mass"); 
    hist_lep_pt[i]->SetNameTitle("hist_lep_pt", "Lepton pT"); 
    hist_met[i]->SetNameTitle("hist_met", "Missing ET");
    hist_mll[i]->SetBins(20,0,500); 
    hist_lep_pt[i]->SetBins(20,0,1000);
    hist_met[i]->SetBins(20,0,500); 
}

In [13]:
TLorentzVector l1, l2, dileptons; 

In [25]:
for(const auto & i:dataset_IDs){
    hist_mll[i]->Reset(); 
    hist_lep_pt[i]->Reset(); 
    hist_met[i]->Reset();
}

In [15]:
int nentries = (Int_t)dataset->GetEntries();

### 2.1 Fill the histograms 
We can now loop over all events in our dataset, implement desired cuts, and fill the histograms we created above. In this example we choose only events containing exactly to same flavour leptons with opposite charge (i.e. $e^+e^-$ or $\mu^+\mu^-$). 

In [26]:
for (int i = 0; i < nentries ; i++){
    
    //if( i%1000000 == 0){ cout << i << " events processed" << endl;}    
    if( i % 1000 == 0 ){ 
    dataset->GetEntry(i); // We "pull out" the i'th entry in the chain. The variables are now 
                          // available through the names we have given them. 
    
    // Cut #1: Require (exactly) 2 leptons
    if(lepton_n == 2)
    {
        // Cut #2: Require opposite charge
        if(lepton_charge[0] != lepton_charge[1])
        {
            // Cut #3: Require same flavour (2 electrons or 2 muons)
            if(lepton_type[0] == lepton_type[1])
            {
                l1.SetPtEtaPhiE(lepton_pt[0]/1000., lepton_eta[0], lepton_phi[0], lepton_E[0]/1000.);
                l2.SetPtEtaPhiE(lepton_pt[1]/1000., lepton_eta[1], lepton_phi[1], lepton_E[1]/1000.);
                // Variables are stored in the TTree with unit MeV, so we need to divide by 1000 
                // to get GeV, which is a more practical unit. 
                
                dileptons = l1 + l2;   
    
                hist_mll[ID]->Fill(dileptons.M());
                hist_lep_pt[ID]->Fill(l1.Pt());
                hist_lep_pt[ID]->Fill(l2.Pt()); 
                hist_met[ID]->Fill(MET/1000);   
                
            }
        }
    }
    }        
}
cout << "Loop finished!" << endl; 

Loop finished!


We have now done the "heavy lifting" of an analysis, i.e. looping through all the events. Usually in such an analysis we create new ROOT files where we store the histograms we made above, and then analyse the output in a separate program/script. The advantage of doing this is that you can do the rest of the analysis in another language, e.g. Python, since we are done with part that requires the speed of C$++$. If you want to write ROOT files you can check out the [TFile](https://root.cern.ch/doc/master/classTFile.html) class reference. In this example we will however carry on in C$++$. 

## 3. Scale and classify the histograms

Before we are ready to make plots we need to do some further processing of the histograms we made above. A very important task is to scale the histograms to the right cross section and luminosity, and we also need group the histograms in different background categories.   

[THStack](https://root.cern.ch/doc/master/classTHStack.html) 
[TColor](https://root.cern.ch/doc/master/classTColor.html) 

In [34]:
map<TString, vector<Int_t> > Background_IDs; 

In [36]:
Background_IDs["Diboson"] = {105985, 105986, 105987};
Background_IDs["Zjets"] = {147770, 147771, 147772};
Background_IDs["ttbar"] = {117049, 117050};
Background_IDs["singleTop"] = {110090, 110091, 110119, 110140};
Background_IDs["Wjets"] = {167740, 167741, 167742, 167743, 167744, 167745, 167746, 167747, 167748};
Background_IDs["DY"] = {173041, 173042, 173043, 173044, 173045, 173046};
Background_IDs["Higgs"] = {160155, 160205, 161005, 161055}; 

In [33]:
map<TString, TH1*> Histograms;  

In [40]:
vector<TString> Background; 

In [41]:
Background = {"Diboson", "Zjets", "ttbar", "singleTop", "Wjets", "DY", "Higgs"}; 

In [42]:
for(const auto i:Background){
    Histograms[i] = new TH1F(); 
}

In [27]:
map<TString, Int_t> colors; 

In [28]:
colors["Diboson"] = kAzure+1; 
colors["Zjets"] = kOrange+2; 
colors["ttbar"] = kRed-5;
colors["singleTop"] = kRed+2; 
colors["Wjets"] = kGray; 
colors["DY"] = kOrange+1; 
colors["Higgs"] = kViolet-9; 

In [ ]:
for(const auto h:Background){
    hist_mll[h]->SetFillColor(colors[h]); 
    hist_met[h]->SetFillColor(colors[h]);
    hist_lep_pt[h]->SetFillColor(colors[h]);
    
    hist_mll[h]->SetLineColor(colors[h]); 
    hist_met[h]->SetLineColor(colors[h]);
    hist_lep_pt[h]->SetLineColor(colors[h]);
}

In [3]:
ifstream info("Infofile.txt"); 

In [4]:
map<Int_t, Double_t> scale_factors; 

In [5]:
TString process; 
Int_t DSID; 
Int_t n_events; 
Double_t red_eff; 
Double_t sum_w; 
Double_t x_sec; 

In [6]:
Double_t L = 1000.6; // Luminosity (pb)

In [7]:
while(info >> process >> DSID >> n_events >> red_eff >> sum_w >> x_sec){
    scale_factors[DSID] = x_sec*L/n_events; 
}

## 4. Stacking and plotting 

In [22]:
THStack *stack_mll = new THStack("Invariant mass", "");

In [23]:
THStack *stack_met = new THStack("Missing ET", ""); 

In [24]:
for(const auto h:bgs){
    stack_mll->RecursiveRemove(hist_mll[h]); // Remove previously stacked histograms  
    stack_met->RecursiveRemove(hist_met[h]);
    stack_mll->Add(hist_mll[h]); 
    stack_met->Add(hist_met[h]);
}    

Now we make a legend with the different backgrounds, and plot the stacks. 

In [25]:
gStyle->SetLegendBorderSize(0); // Remove (default) border around legend 
TLegend *legend = new TLegend(0.65, 0.70, 0.85, 0.85); 

In [26]:
legend->Clear();
for(const auto i:bgs){
    legend->AddEntry(hist_mll[i], i, "f");  // Add your histograms to the legend
} 

In [27]:
TCanvas *C = new TCanvas("c", "c", 600, 600);

In [28]:
gPad->SetLogy(); // Set logarithmic y-axis

In [29]:
stack_mll->Draw(); 
stack_mll->GetYaxis()->SetTitle("# events");
stack_mll->GetYaxis()->SetTitleOffset(1.3); 
stack_mll->GetXaxis()->SetTitle("m_{ll} (GeV)");
stack_mll->GetXaxis()->SetTitleOffset(1.3);
legend->Draw();
C->Draw();

In [30]:
stack_met->Draw(); 
stack_met->GetYaxis()->SetTitle("# events");
stack_met->GetYaxis()->SetTitleOffset(1.3); 
stack_met->GetXaxis()->SetTitle("E_{T}^{miss} (GeV)");
stack_met->GetXaxis()->SetTitleOffset(1.3);
stack_met->GetXaxis()->SetLimits(0,250);
legend->Draw();
C->Draw();

## 5. Statistical analysis of results? 